## Advent of code 2019 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.8-v7.3.7
# to activate it from a git bash shell: source .venv/Scripts/activate
# to generate its requirements: pip freeze > .venv-requirements.txt
# to re-install from requirements: python -m venv .venv; source .venv/Scripts/activate; pip install -r .venv-requirements.txt
# (may need the full path to python, e.g. ~/AppData/Local/Programs/Python/Python310/python.exe)

import collections
import itertools
import re
import copy
import math
import sys
import time
#import json
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 2
# start_ts=1643360000
# mv ~/Downloads/input data_src/2019-day-2-input.txt
# big input file looks like: about 100 numbers
# idea: part 1 parse as single list, then simply run

sample1='''
1,9,10,3,2,3,11,0,99,30,40,50
'''

sample2='''
1,1,1,4,99,5,6,0,99
'''

def run_opcodes(data):
    i=0
    while True:
        if data[i]==1:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a+b
            i+=4
        elif data[i]==2:
            a=data[data[i+1]]
            b=data[data[i+2]]
            data[data[i+3]]=a*b
            i+=4
        elif data[i]==99:
            break
        else:
            assert False

sample1=open('data_src/2019-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ]
for noun in range(100):
    for verb in range(100):
        data=list(data0)
        data[1]=noun
        data[2]=verb
        run_opcodes(data)
        if data[0]==19690720:
            answer=100*noun+verb
            print(f'{answer=}')
            exit()

In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed